In [6]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.7 MB/s eta 0:00:00


In [2]:
display_architecture=False
import torch
import json

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [9]:
model = T5ForConditionalGeneration.from_pretrained('t5-large')
tokenizer = T5Tokenizer.from_pretrained('t5-large')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:220: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes

# Processing Unit only CPU

In [10]:
device = torch.device('cpu')

In [13]:
if display_architecture == True:
  print(model.config)

# Encoder Details

In [21]:
model.encoder

T5Stack(
  (embed_tokens): Embedding(32128, 1024)
  (block): ModuleList(
    (0): T5Block(
      (layer): ModuleList(
        (0): T5LayerSelfAttention(
          (SelfAttention): T5Attention(
            (q): Linear(in_features=1024, out_features=1024, bias=False)
            (k): Linear(in_features=1024, out_features=1024, bias=False)
            (v): Linear(in_features=1024, out_features=1024, bias=False)
            (o): Linear(in_features=1024, out_features=1024, bias=False)
            (relative_attention_bias): Embedding(32, 16)
          )
          (layer_norm): T5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): T5LayerFF(
          (DenseReluDense): T5DenseActDense(
            (wi): Linear(in_features=1024, out_features=4096, bias=False)
            (wo): Linear(in_features=4096, out_features=1024, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
            (act): ReLU()
          )
          (layer_norm): T5LayerNorm()
 

In [16]:
model.decoder

T5Stack(
  (embed_tokens): Embedding(32128, 1024)
  (block): ModuleList(
    (0): T5Block(
      (layer): ModuleList(
        (0): T5LayerSelfAttention(
          (SelfAttention): T5Attention(
            (q): Linear(in_features=1024, out_features=1024, bias=False)
            (k): Linear(in_features=1024, out_features=1024, bias=False)
            (v): Linear(in_features=1024, out_features=1024, bias=False)
            (o): Linear(in_features=1024, out_features=1024, bias=False)
            (relative_attention_bias): Embedding(32, 16)
          )
          (layer_norm): T5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): T5LayerCrossAttention(
          (EncDecAttention): T5Attention(
            (q): Linear(in_features=1024, out_features=1024, bias=False)
            (k): Linear(in_features=1024, out_features=1024, bias=False)
            (v): Linear(in_features=1024, out_features=1024, bias=False)
            (o): Linear(in_features=1024, out_feat

In [20]:
model.forward

<bound method T5ForConditionalGeneration.forward of T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4

In [32]:
def sum(t, l): # t for text , l for max length
  pre_text = t.strip().replace('\n','')
  t5_text = "summarize: " + pre_text
  print("Prep Text \n",t5_text)
  token_text = tokenizer.encode(t5_text,return_tensors="pt").to(device)
  summary_ids = model.generate(token_text,
                               num_beams=4,
                               no_repeat_ngram_size =2,
                               max_length=l,
                               early_stopping=True)
  return tokenizer.decode(summary_ids[0],skip_special_tokens=True)


## Stories & Books Snippets Summarization

In [40]:
text ="""
Rhymes from the Russian" is a collection of poems by American poet and translator Gertrude Stein. The book was published in 1912 and is known for its experimental and avant-garde style. In this collection, Stein attempted to capture the essence of Russian poetry through her unique linguistic and stylistic innovations, often using repetition and wordplay to create a sense of rhythm and musicality.
Stein's poems in "Rhymes from the Russian" do not adhere to traditional poetic forms or conventions. Instead, she explores language and meaning in unconventional ways, emphasizing the sounds and patterns of words. While the poems are inspired by Russian literature and culture, they are not direct translations but rather Stein's interpretation and reimagining of Russian themes.
Overall, "Rhymes from the Russian" is a groundbreaking work of modernist poetry that challenges traditional notions of language and poetry, offering readers a unique and experimental exploration of words and their meanings. It is considered a significant contribution to the avant-garde literary movement of the early 20th century.
"""
print("Number of characters:",len(text))
summary=sum(text,200)
print ("\n\nSummarized text: \n",summary)

Number of characters: 1113
Prep Text 
 summarize: Rhymes from the Russian" is a collection of poems by American poet and translator Gertrude Stein. The book was published in 1912 and is known for its experimental and avant-garde style. In this collection, Stein attempted to capture the essence of Russian poetry through her unique linguistic and stylistic innovations, often using repetition and wordplay to create a sense of rhythm and musicality.Stein's poems in "Rhymes from the Russian" do not adhere to traditional poetic forms or conventions. Instead, she explores language and meaning in unconventional ways, emphasizing the sounds and patterns of words. While the poems are inspired by Russian literature and culture, they are not direct translations but rather Stein's interpretation and reimagining of Russian themes.Overall, "Rhymes from the Russian" is a groundbreaking work of modernist poetry that challenges traditional notions of language and poetry, offering readers a unique and ex

In [41]:
text ="""
Dr. Emily Thornton, a brilliant physicist, invents a time machine.
She is faced with a difficult decision: should she use it to change the course of history for
the better, or should she leave the past untouched? As she navigates the complex moral and ethical
dilemmas of time travel, Dr. Thornton learns that altering the past has consequences she could never have imagined.
"""
print("Number of characters:",len(text))
summary=sum(text,100)
print ("\n\nSummarized text: \n",summary)

Number of characters: 380
Prep Text 
 summarize: Dr. Emily Thornton, a brilliant physicist, invents a time machine. She is faced with a difficult decision: should she use it to change the course of history for the better, or should she leave the past untouched? As she navigates the complex moral and ethical dilemmas of time travel, Dr. Thornton learns that altering the past has consequences she could never have imagined.


Summarized text: 
 dr. Emily thornton invents a time machine to change the course of history. she must decide whether to alter the past for the better or leave it untouched, cnn.com's tom charity reviews the book and recommends it for young readers ages 10 and up...


## Corporate Law Summarization

In [39]:
text ="""
In the realm of corporate law, corporate governance and shareholder rights are paramount. Corporate governance refers to the system of rules, practices, and processes by which a company is directed and controlled. It encompasses the relationships among a company's management, its board of directors, its shareholders, and other stakeholders. Effective corporate governance is essential for maintaining the integrity and ethical behavior of corporations.
One crucial aspect of corporate governance is ensuring that shareholders have the rights and mechanisms to participate in the decision-making processes of the corporation. Shareholders' rights include the ability to vote on key issues, access to information about the company's operations, and the opportunity to hold the company's management and directors accountable for their actions.
Corporate law also addresses the duty of care and duty of loyalty owed by directors and officers of a corporation. Directors are expected to act in the best interests of the corporation and its shareholders, making informed decisions and avoiding conflicts of interest. The duty of loyalty requires directors and officers to put the interests of the corporation before their personal interests.
Additionally, corporate law regulates mergers and acquisitions, securities offerings, and various financial transactions. It sets forth the legal framework for forming and dissolving corporations, as well as rules governing the distribution of dividends and the protection of minority shareholders.
In the global business landscape, international corporate law is of increasing importance. Multinational corporations must navigate complex legal regimes in different jurisdictions, often facing issues related to cross-border mergers, antitrust regulations, and international trade laws.
The dynamic nature of corporate law requires practitioners to stay informed about evolving regulations, court decisions, and industry best practices. Successful corporate lawyers possess a deep understanding of corporate governance principles, financial markets, and the legal intricacies of business transactions.
Summarizing this extensive content would require your LLM summarizer to distill the key points related to corporate governance, shareholder rights, director and officer duties, mergers and acquisitions, and international corporate law, while maintaining accuracy and relevance.
"""
print("Number of characters:",len(text))
summary=sum(text,1000)
print ("\n\nSummarized text: \n",summary)

Number of characters: 2419
Prep Text 
 summarize: In the realm of corporate law, corporate governance and shareholder rights are paramount. Corporate governance refers to the system of rules, practices, and processes by which a company is directed and controlled. It encompasses the relationships among a company's management, its board of directors, its shareholders, and other stakeholders. Effective corporate governance is essential for maintaining the integrity and ethical behavior of corporations.One crucial aspect of corporate governance is ensuring that shareholders have the rights and mechanisms to participate in the decision-making processes of the corporation. Shareholders' rights include the ability to vote on key issues, access to information about the company's operations, and the opportunity to hold the company's management and directors accountable for their actions.Corporate law also addresses the duty of care and duty of loyalty owed by directors and officers of a corpora